# Walls to thermal circuit

This notebook presents how the _description of the building_, given in a folder that contains two types of files:
- types of walls (thermo-physical properties)
- data on each walls

is transformed in _disassembled thermal networks_, each one characterized by the matrices A, G, C and the vectors b, f, y.

In [1]:
import pandas as pd
import pd_dm4bem

The input data needed for the thermal model of the building is in a folder.

In [2]:
folder = "bldg_wall2TC"

## Wall types

Let's consider two types of walls described in the file `wall_types.csv`.

In [3]:
wall_types = pd.read_csv(folder + '/wall_types.csv')
wall_types

,type,Material,Conductivity,Specific heat,Density,Width,Mesh
0,0,Concrete,1.400,880,2300,0.200,1
1,0,Insulation,0.027,1210,55,0.080,2
2,1,Glass,1.400,750,2500,0.004,1


The file `wall_types.csv` contains:
- the physical properties of the materials given in SI units: __conductivity__ in W·m⁻¹·K⁻¹, __specific heat__ in J·kg⁻¹·K⁻¹, __density__ in kg·m⁻³;
- the __width__ of the layers in m;
- the number of __meshes__ in which each layer is numerically discretized; each mesh has two resistances and a capacity.

There are two types of walls in `wall_types.csv`:

- The walls of __type 0__ have two layers: _Concrete_ and _Insulation_. _Concrete_ layer has one mesh (two resistances and a capacity) and _Insulation_ layer has two meshes (i.e., four resistances and two capacities).
- The walls of __type 1__ has one layer: _Glass_, discretized in one mesh.

![wall_types](./bldg_wall2TC/wall_types.svg)
> Figure 1. The two types of wall described in file `wall_types.csv`

## Walls data
The models of walls are based on the materials given in `wall_types.cvs`. For each specific wall, the area surface and the information needed for the boundary conditions is given.

There are three kinds of walls:
- generic,
- outdoor,
- indoor.

The `walls_generic.csv`, `walls_out.csv`, and `walls_in.csv` files contain the characteristics of the surfaces of the walls (Table 1).

>Table 1. Columns of `walls_generic.csv`, `walls_out.csv`, and `walls_in.csv` files. The columns that are present in the file are maked by ✓ for _generic, _out and _in walls.

|Column|Unit|Definition                             | Example|_generic|_out|_in|
|------|----|---------------------------------------|--------|--------|---|---|
| ID   |    | Identifiant of each wal               | w2     | ✓      | ✓ | ✓ |
| type |    | Type of wall given in `wall_types.csv`| 1      | ✓      | ✓ | ✓ |
| Area | m² | Surface area of the wall              | 25     | ✓      | ✓ | ✓ |
| β    | °  | Tilt angle: 90° vertical; >90° upward facing| 90 | ✓  | ✓ |   |
| γ    | °  | Azimuth: 0° South, 180° North, >0° westward | 90| ✓      | ✓ |   |
| T0   | °C | Temperature source of outer surface   | To     | ✓      | ✓ |   |
| T1   | °C | Temperature source of inner surface   | Ti     | ✓      |   |   |
| Q0   | W  | Flow rate source on outer surface     | Qo     | ✓      | ✓ | ✓ |
| Q1   | W  | Flow rate source on inner surface     | Qi     | ✓      | ✓ | ✓ |
| h0   | W·m⁻²·K⁻¹| Convection coefficient of outer surface | 25 | ✓  | ✓ | ✓ |
| h1   | W·m⁻²·K⁻¹| Convection coefficient of inner surface |  8 | ✓  | ✓ | ✓ |
| α0   |    | Short-wave absorbtion coefficient of outer surface | 0.25 | ✓ | ✓ | ✓ |
| α1   |    | Short-wave absorbtion coefficient of inner surface | 0.30 | ✓ | ✓ | ✓ |
| ε0   |    | Long-wave hemispherical emissivity of outer surface| 0.85 | ✓ | ✓ | ✓ |
| ε1   |    | Long-wave hemispherical emissivity of inner surface| 0.70 | ✓ | ✓ | ✓ |
| y    |    | Output temperature nodes (by using slicing) |[0, -1] | ✓ | ✓ | ✓ |

__Notes__:
1. The _generic_ walls may characterized _out_ or _in_ walls by not specifying temperare sources (see `wall_generic.csv`, `wall_out.csv`, and `wall_in.csv` files for examples).

2. The _out_ walls do not need the temperature sources on the inner surfaces, _T1_.

3. The _in_ walls do not need tilt (β) and azimuth (γ) angles and the temperature sources on the outer (_T0_) and the inner (_T1_) surfaces.

### Generic wall

Generic walls contain all the information listed in Table 1. For example, `wall_generic.csv` file describes the walls shown in Figure 2.

In [4]:
walls = pd.read_csv(folder_path + "/walls_generic.csv")
walls

NameError: name 'folder_path' is not defined

The file `walls_generic.csv` describes four walls (Figure 2):
- according to the types of walls:
    - w0 and w1 are of __type__ 1, i.e., concrete and insulation;
    - w2 and w3 are of __type__ 0, i.e., glass.
- according to the temperature sources:
    - w0 and w1 are outdoor walls that have as boundary condition the outdoor temperature __T0__; the temperature on the inner surface is a room temperature, which is an output of the model.
    - w1 is a wall that has temperature sources on both sides.
    - w3 is an inner wall that separated two rooms. it does not need information for solar direct radiation (tilt and azimuth angles).

Outdoor walls, such as gw0 (generic wall w0) and gw2 (generic wall w2) in Figure 2, can be obtained from the generic walls by not considering a temperature source __T1__ on inner surface, which is indicated by NaN in rows w0 and w2 for column __T1__.

Likewise, indoor walls, such as gw3 (generic wall 3) in Figure 2, can be obtained from the generic walls by not indicating there are no temperature sources in __T0__ and __T1__, i.e. `T0 = NaN` and `T1 = NaN`.

The outputs __y__ are indicated in a list using [slicing](https://python-reference.readthedocs.io/en/latest/docs/brackets/slicing.html). For example, the outputs of wall w0 are temperature nodes 0 and 3 (the last node).

The numbering convention of the elements of the circuits is:
- the nodes (i.e., temperatures) are numbered from left to right and from up to down;
- the oriented branches (i.e., flows) are numbered from the node with a lower index to the node with a higher index;
- the __reference__ node (temperature) is not numbered.

![generic_wall](./bldg_wall2TC/walls_generic.svg)
> Figure 2. Four examples of generic walls described in the file `wall_generic.csv`. Walls gw0 and gw1 are of __type 1__ (glass), described in `wall_type.csv`. Walls gw2 and gw3 are of __type 0__ concrete and insulation), described in `wall_type.csv`.

The files describing the types (`wall_types.csv`) and the data (`walls_generic.csv`) of the walls contain the information needed to obtain the disassembled thermal circuits. The names of the walls can have a prefix. 

In [ ]:
# Thermal circuits from wall types and wall data
TCd_generic = pd_dm4bem.wall2TC(wall_types, walls, prefix="g")

Each matrix (A, G, C) of vector (b, f, y) of the thermal circuit can be accessed as Pandas dataframes.

In [ ]:
TCd_generic['gw0']['A']

All elements of the thermal circuit can be printed.

In [ ]:
pd_dm4bem.print_TC(TCd_generic['gw0'])

All thermal circuits read from the file `walls_generic.csv` can be printed.

In [ ]:
print('TCd_generic')
for key in TCd_generic.keys():
    print('Wall:', key)
    pd_dm4bem.print_TC(TCd_generic[key])

### Outdoor walls

Outdoor walls do not have temperature sources on indoor surfaces (Figure 3). They can be obtained from generic walls by not indicating temperature sources T0 and T1 or they can be described by specific files, such as `walls_in.csv`.

Figure 3a shows a wall of __type__ 1 (described in `wall_types.csv`) with only one layer which has one mesh.

Figure 3b shows a wall of __type__ 0 (described in `wall_types.csv`) with two layers (concrete and insulation). 

![in_wall](./bldg_wall2TC/walls_out.svg)
> Figure 3. Outdoor walls: a) of __type__ 1 b) of __type__ 0. The __type__ is indicated in the file `wall_types.csv`.

In [ ]:
walls = pd.read_csv(folder_path + '/walls_in.csv')
walls

In [ ]:
# Thermal circuits from data & type files of walls
TCd_out = pd_dm4bem.wall2TC(wall_types, walls, prefix="o")

# Uncomment below to print all thermal circuits
# print('TCd_out')
# for key in TCd_out.keys():
#    print('Wall:', key)
#    pd_dm4bem.print_TC(TCd_out[key])

### Indoor walls

Indoor walls do not have temperature sources on surfaces (Figure 4). They can be obtained from generic walls by not indicating temperature sources T0 and T1 or they can be described by specific files, such as `walls_in.csv`.

Figure 4a shows a wall of __type__ 1 (described in `wall_types.csv`) with only one layer which has one mesh.

Figure 4b shows a wall of __type__ 0 (described in `wall_types.csv`) with two layers (concrete and insulation); the insulation layer is discretized in two meshes.

![in_wall](./bldg_wall2TC/walls_in.svg)
> Figure 4. Indoor walls: a) of __type__ 1 b) of __type__ 0. The __type__ is indicated in the file `wall_types.csv`.

In [ ]:
walls = pd.read_csv(folder_path + '/walls_in.csv')
walls

In [ ]:
# Thermal circuits from data & type files of walls
TCd_in = pd_dm4bem.wall2TC(wall_types, walls, prefix="i")

# print('TCd_in')
# for key in TCd_in.keys():
#    print('Wall:', key)
#    pd_dm4bem.print_TC(TCd_in[key])